In [1]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [2]:
!pip install transformers[torch] datasets torch peft trl==0.4.7 bitsandbytes accelerate cuda-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00


In [13]:
!zip -r ./lora_model_2.zip ./lora_model
# pass

  adding: lora_model/ (stored 0%)
  adding: lora_model/training_args.bin (deflated 51%)
  adding: lora_model/adapter_config.json (deflated 49%)
  adding: lora_model/tokenizer_config.json (deflated 68%)
  adding: lora_model/tokenizer.json (deflated 74%)
  adding: lora_model/adapter_model.safetensors (deflated 8%)
  adding: lora_model/README.md (deflated 66%)
  adding: lora_model/special_tokens_map.json (deflated 79%)


In [4]:
# !unzip ./lora_model.zip
# pass

In [3]:
import torch
torch.cuda.empty_cache()

In [4]:
# Init imports and constants

import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, AutoTokenizer, pipeline
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig, PeftModel, get_peft_model

BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
dataset = load_dataset("Amod/mental_health_counseling_conversations", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
# Loading tokenizer

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [6]:
# Loading model and formatting dataset

quant_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    quantization_config=quant_config,
    trust_remote_code=True
)

def merge_columns(example):
  messages = [
      {
          "role": "user",
          "content": example["Context"]
      },
      {
          "role": "assistant",
          "content": example["Response"]
      }
  ]

  example["prediction"] = messages
  return example

predictions = dataset.map(merge_columns)
formatted_dataset = predictions.map(lambda row: {"formatted": tokenizer.apply_chat_template(row["prediction"], tokenize=False, add_generation_prompt=False)})

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [9]:
print(base_model.get_memory_footprint() / 1**9)

758307584.0


In [7]:
# LoRa configuration

lora_config = LoraConfig(
    r=4,
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

lora_model = get_peft_model(base_model, lora_config)

In [12]:
# Train LoRa model

training_arguments = TrainingArguments(
    output_dir="training_output",
    num_train_epochs=1,
    logging_steps=100,
    save_steps=100,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    warmup_steps=100,
)

trainer = SFTTrainer(
    model=lora_model,
    train_dataset=formatted_dataset,
    dataset_text_field="formatted",
    args=training_arguments
)

trainer.train()
trainer.save_model("./lora_model")

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
100,2.445700
200,2.303300
300,2.243300
400,2.222800
500,2.183400
600,2.190000
700,2.164300
800,2.155700
900,2.190200
1000,2.190100


Step,Training Loss
100,2.445700
200,2.303300
300,2.243300
400,2.222800
500,2.183400
600,2.190000
700,2.164300
800,2.155700
900,2.190200
1000,2.190100


In [14]:
# Merge LoRa and base model to create fine tuned model

full_base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    trust_remote_code=True,
)

lora_adapter = PeftModel.from_pretrained(
    full_base_model,
    "./lora_model"
)

fine_tuned_model = lora_adapter.merge_and_unload()

fine_tuned_model.save_pretrained("./fine_tuned_model_therapist")
tokenizer.save_pretrained("./fine_tuned_model_therapist")

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/tokenizer.json')

In [15]:
# Testing original model

text_gen = pipeline(
    "text-generation",
    model=BASE_MODEL,
)

messages = [
    {
        "role": "user",
        "content": "I'm going through a tough time, what can i do to feel better?",
    },
]

prompt = text_gen.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
result = text_gen(prompt)
print(result[0]["generated_text"])

<|user|>
I'm going through a tough time, what can i do to feel better?</s>
<|assistant|>
I understand that you are going through a tough time. Here are some suggestions that may help you feel better:

1. Take care of yourself: make sure you are getting enough sleep, eating a healthy diet, and exercising regularly.

2. Practice self-care: take a relaxing bath, read a book, or spend time with loved ones.

3. Connect with others: reach out to friends or family members who can provide emotional support.

4. Practice mindfulness: focus on the present moment and let go of worries and negative thoughts.

5. Get enough exercise: engage in physical activity that you enjoy, such as walking, cycling, or swimming.

6. Practice gratitude: focus on the things you are grateful for in your life.

7. Seek professional help: if you are struggling with your mental health, consider seeking professional help from a mental health professional.

remember, it's okay to ask for help and support. Don't hesitate

In [16]:
# Testing fine tuned model

fine_text_gen = pipeline(
    "text-generation",
    model=fine_tuned_model,
    tokenizer=tokenizer,
    max_new_tokens=200
)

messages = [
    {
        "role": "user",
        "content": "I'm going through a tough time, what can i do to feel better?",
    },
]

prompt = fine_text_gen.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
result = fine_text_gen(prompt)
print(result[0]["generated_text"])

<|user|>
I'm going through a tough time, what can i do to feel better?</s>
<|assistant|>
I'm glad you're here to ask for help. I'm glad you're feeling the way you are. I'm glad you're feeling the way you are. I'm glad you're feeling the way you are. I'm glad you're feeling the way you are. I'm glad you're feeling the way you are. I'm glad you're feeling the way you are. I'm glad you're feeling the way you are. I'm glad you're feeling the way you are. I'm glad you're feeling the way you are. I'm glad you're feeling the way you are. I'm glad you're feeling the way you are. I'm glad you're feeling the way you are. I'm glad you're feeling the way you are. I'm glad you're feeling the way you are. I'm glad you
